In [40]:
"""
Extract TreeMap (c.a. 2016) statistics in active fire detections

"""

import os, sys
import ee
import geemap
import time
import pandas as pd

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()

ee.Initialize(project='jfsp-aspen')

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Success")

Success


In [4]:
# Load the USFS TreeMap
treemap = ee.ImageCollection("USFS/GTAC/TreeMap/v2016")
print(f"TreeMap bands available for analysis:\n\n{treemap.first().bandNames().getInfo()}")

TreeMap bands available for analysis:

['ALSTK', 'BALIVE', 'CANOPYPCT', 'CARBON_D', 'CARBON_DWN', 'CARBON_L', 'DRYBIO_D', 'DRYBIO_L', 'FLDSZCD', 'FLDTYPCD', 'FORTYPCD', 'GSSTK', 'QMD_RMRS', 'SDIPCT_RMRS', 'STANDHT', 'STDSZCD', 'TPA_DEAD', 'TPA_LIVE', 'Value', 'VOLBFNET_L', 'VOLCFNET_D', 'VOLCFNET_L']


In [54]:
class_codes = treemap.first().get('FORTYPCD_class_values').getInfo()
class_names = treemap.first().get('FORTYPCD_class_names').getInfo()
code_to_name = dict(zip(class_codes, class_names))
species_df = pd.DataFrame(list(code_to_name.items()), columns=['species_code', 'species_name'])
species_df['species_code'] = species_df['species_code'].astype(int)
species_df.head()

,species_code,species_name
0,101,Jack pine
1,102,Red pine
2,103,Eastern white pine
3,104,Eastern white pine / eastern hemlock
4,105,Eastern hemlock


In [20]:
# Save this file out.
out_fp = os.path.join(projdir,'data/tabular/mod/treemap_fortypcd_species_mapping.csv')
df = pd.DataFrame(list(code_to_name.items()), columns=['FORTYPCD', 'SpeciesName'])
print(f"Dictionary saved to {out_fp}")

Dictionary saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/tabular/mod/treemap_fortypcd_species_mapping.csv


In [41]:
# Load the Active Fire Detections (AFD)
afds = ee.FeatureCollection('projects/jfsp-aspen/assets/AFD/combined-afd_aspen-fires_2018_to_2023')
afds = afds.filter(ee.Filter.eq('na_l3name', 'Southern Rockies'))
print(afds.size().getInfo())
print(afds.first().propertyNames().getInfo())

77339
['fid', 'FRP', 'NIFC_NAME', 'LONGITUDE', 'ACQ_YEAR', 'ACQ_TIME', 'SCAN', 'BRIGHTNESS', 'CONFIDENCE', 'TYPE', 'SATELLITE', 'na_l3name', 'DAYNIGHT', 'TRACK', 'INSTRUMENT', 'VID', 'NIFC_ID', 'START_YEAR', 'ACQ_DATETI', 'ACQ_MONTH', 'WF_CESSATI', 'VERSION', 'BRIGHT_T31', 'DISCOVERY_', 'afdID', 'LATITUDE', 'system:index', 'ACQ_DATE']


## Calculate species-specific cover, live basal area (mean), and canopy percent (mean)

In [42]:
# Clean the property names before exporting reductions (just keep the ID)
afds_ = afds.select(['afdID'])

In [46]:
def species_histogram(ftr):
    hist = imageC.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=ftr.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('FORTYPCD')

    hist_dict = ee.Dictionary(hist)
    flat_ftr = hist_dict.keys().iterate(
        lambda key, f: ee.Feature(f).set(ee.String(key), hist_dict.get(key)), ftr
    )
    return ee.Feature(flat_ftr)


# Apply this to the AFDs
fortypcd = afds_.map(species_histogram)

# Print a sample
sample = fortypcd.limit(1).getInfo()
properties = sample['features'][0]['properties']
df = pd.DataFrame([properties])
print("Sample DataFrame:")
print(df.head())

# Export to Asset/Drive
fortypcd = fortypcd.map(lambda ftr: ftr.setGeometry(None)) # drop geometry column

task = ee.batch.Export.table.toDrive(
    collection=fortypcd,
    description='afd_aspen-fires_treemap-fortypcd',
    fileFormat='CSV',
    folder='TreeMap'
)
# Start the export task
task.start()
print("Export to Google Drive started!")
# Monitor the task until it's finished
monitor_export(task, timeout=120) # print every X seconds

Sample DataFrame:
   182  185        201         221        261  265  266        267      281  \
0   28   23  40.262745  146.698039  27.286275    4    1  85.882353  9.25098   

   365  366        369         901       971        999       afdID  
0    6    3  12.035294  192.686275  19.25098  25.435294  MODIS12004  
Export to Google Drive started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Export completed successfully !!!!


In [ ]:
# Species BALIVE and CANOPYPCT

In [125]:
def species_metrics(ftr):
    # Get species histogram
    hist = imageC.select('FORTYPCD').reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=ftr.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('FORTYPCD')

    hist_dict = ee.Dictionary(hist)
    
    # Initialize feature with species histogram keys and calculate averages
    def add_species_metrics(key, f):
        key_str = ee.String(key)
        # Mask the image by the species type
        masked_image = imageC.updateMask(imageC.select('FORTYPCD').eq(ee.Number.parse(key).toInt()))
        
        # Calculate average BALIVE and CANOPYPCT for the masked image within the polygon
        metrics = masked_image.select(['BALIVE', 'CANOPYPCT']).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ftr.geometry(),
            scale=30,
            maxPixels=1e13
        )
        
        # Add the averages to the feature with species-specific prefix
        f = ee.Feature(f).set(key_str.cat('_BALIVE_mean'), metrics.get('BALIVE'))
        f = f.set(key_str.cat('_CANOPYPCT_mean'), metrics.get('CANOPYPCT'))
        return f

    # Apply species metrics calculation to each species type in histogram
    flat_ftr = hist_dict.keys().iterate(add_species_metrics, ftr)
    
    return ee.Feature(flat_ftr)

fortypcd_metrics = afds_.map(species_metrics)

# Remove geometry column first
fortypcd_metrics = fortypcd_metrics.map(lambda ftr: ftr.setGeometry(None)) # drop geometry column

export_task = ee.batch.Export.table.toDrive(
    collection=fortypcd_metrics,
    description='afd_aspen-fires_treemap-balive_canopypct',
    fileFormat='CSV', 
    fileNamePrefix='afd_aspen-fires_treemap-balive_canopypct',
    folder='TreeMap'
)

# Start the export task
export_task.start()
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task, 120) # print every 120 seconds

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting fo

In [ ]:
# Load the exported CSV files.

In [126]:
# Forest Type
fp = os.path.join(projdir,'data/earth-engine/exports/treemap/afd_aspen-fires_treemap-fortypcd.csv')
fortypcd = pd.read_csv(fp)
fortypcd.drop(columns=['system:index','.geo'], inplace=True)
fortypcd.columns

Index(['182', '185', '201', '221', '261', '265', '266', '267', '281', '365',
       '366', '369', '901', '971', '999', 'afdID'],
      dtype='object')

In [127]:
# Pivot longer and join to get species name
fortypcd_l = fortypcd.melt(id_vars=['afdID'], var_name='species_code', value_name='count')
fortypcd_l['species_code'] = fortypcd_l['species_code'].astype(int)
# Drop 'NaN' counts
fortypcd_l.dropna(subset=['count'], inplace=True)
# Check on the results
fortypcd_l = fortypcd_l.sort_values(by=['afdID'])
fortypcd_l.head()

,afdID,species_code,count
560398,MODIS10003,267,17.870588
637737,MODIS10003,281,151.721569
947093,MODIS10003,901,79.211765
483059,MODIS10003,266,1.000000
405720,MODIS10003,265,2.000000


In [128]:
fortypcd_l = fortypcd_l.merge(species_df, on='species_code', how='left')
fortypcd_l = fortypcd_l[['afdID','species_code','species_name','count']]
fortypcd_l['count'] = fortypcd_l['count'].round().astype(int)
fortypcd_l.head()

,afdID,species_code,species_name,count
0,MODIS10003,267,Grand fir,18
1,MODIS10003,281,Lodgepole pine,152
2,MODIS10003,901,Aspen,79
3,MODIS10003,266,Engelmann spruce / subalpine fir,1
4,MODIS10003,265,Engelmann spruce,2


In [129]:
# Calculate the total forest count and percent cover for each species
total_counts = fortypcd_l.groupby('afdID')['count'].sum().reset_index()
total_counts = total_counts.rename(columns={'count': 'total_count'})

# Join back to the data frame
fortypcd_l_pct = fortypcd_l.merge(total_counts, on='afdID', how='left')
fortypcd_l_pct['pct_cover'] = (fortypcd_l_pct['count'] / fortypcd_l_pct['total_count']) * 100
fortypcd_l_pct.head(12)

,afdID,species_code,species_name,count,total_count,pct_cover
0,MODIS10003,267,Grand fir,18,916,1.965066
1,MODIS10003,281,Lodgepole pine,152,916,16.593886
2,MODIS10003,901,Aspen,79,916,8.624454
3,MODIS10003,266,Engelmann spruce / subalpine fir,1,916,0.109170
4,MODIS10003,265,Engelmann spruce,2,916,0.218341
5,MODIS10003,201,Douglas-fir,120,916,13.100437
6,MODIS10003,185,Pinyon / juniper woodland,8,916,0.873362
7,MODIS10003,261,White fir,7,916,0.764192
8,MODIS10003,182,Rocky Mountain juniper,1,916,0.109170
9,MODIS10003,971,Deciduous oak woodland,22,916,2.401747


In [130]:
# BALIVE and CANOPYPCT
fp = os.path.join(projdir,'data/earth-engine/exports/treemap/afd_aspen-fires_treemap-balive_canopypct.csv')
stats_df = pd.read_csv(fp)
stats_df.drop(columns=['system:index','.geo'], inplace=True)
stats_df.columns

Index(['182_BALIVE_mean', '182_CANOPYPCT_mean', '185_BALIVE_mean',
       '185_CANOPYPCT_mean', '201_BALIVE_mean', '201_CANOPYPCT_mean',
       '221_BALIVE_mean', '221_CANOPYPCT_mean', '261_BALIVE_mean',
       '261_CANOPYPCT_mean', '265_BALIVE_mean', '265_CANOPYPCT_mean',
       '266_BALIVE_mean', '266_CANOPYPCT_mean', '267_BALIVE_mean',
       '267_CANOPYPCT_mean', '281_BALIVE_mean', '281_CANOPYPCT_mean',
       '365_BALIVE_mean', '365_CANOPYPCT_mean', '366_BALIVE_mean',
       '366_CANOPYPCT_mean', '369_BALIVE_mean', '369_CANOPYPCT_mean',
       '901_BALIVE_mean', '901_CANOPYPCT_mean', '971_BALIVE_mean',
       '971_CANOPYPCT_mean', '999_BALIVE_mean', '999_CANOPYPCT_mean', 'afdID'],
      dtype='object')

In [132]:
metrics_cols = [col for col in df.columns if '_BALIVE_mean' in col or '_CANOPYPCT_mean' in col]
df_long = df.melt(id_vars=['afdID'], value_vars=metrics_cols, 
                  var_name='metric', value_name='value')

In [133]:
df_long['species_code'] = df_long['metric'].str.extract(r'(\d+)_')[0].astype(int)
df_long['metric'] = df_long['metric'].str.extract(r'_(BALIVE|CANOPYPCT)_mean')[0]
df_long.head()

,afdID,metric,value,species_code
0,MODIS12004,BALIVE,29.388272,182
1,MODIS12004,CANOPYPCT,19.714286,182
2,MODIS12004,BALIVE,53.870844,185
3,MODIS12004,CANOPYPCT,21.130435,185
4,MODIS12004,BALIVE,90.011329,201


In [149]:
stats_df_ = df_long.pivot_table(index=['afdID', 'species_code'], columns='metric', values='value').reset_index()
stats_df_['species_code'] = stats_df_['species_code'].astype(int)
stats_df_.head(12)

metric,afdID,species_code,BALIVE,CANOPYPCT
0,MODIS12004,182,29.388272,19.714286
1,MODIS12004,185,53.870844,21.130435
2,MODIS12004,201,90.011329,48.064186
3,MODIS12004,221,94.476456,41.598749
4,MODIS12004,261,127.596281,44.835010
5,MODIS12004,265,146.119877,56.000000
6,MODIS12004,266,46.205299,17.000000
7,MODIS12004,267,130.266962,53.710685
8,MODIS12004,281,58.489634,28.863925
9,MODIS12004,365,51.567299,29.000000


In [148]:
# Join the two
fortypcd_l_pct['afdID'] = stats_df_['afdID'].astype(str)
stats_df_['afdID'] = stats_df_['afdID'].astype(str)
treemap_stats_df = fortypcd_l_pct.merge(stats_df_, on=['afdID','species_code'], how='left')
treemap_stats_df.head(15)

,afdID,species_code,species_name,count,total_count,pct_cover,BALIVE,CANOPYPCT
0,MODIS12004,267,Grand fir,18,916,1.965066,130.266962,53.710685
1,MODIS12004,281,Lodgepole pine,152,916,16.593886,58.489634,28.863925
2,MODIS12004,901,Aspen,79,916,8.624454,60.523708,42.095899
3,MODIS12004,266,Engelmann spruce / subalpine fir,1,916,0.109170,46.205299,17.000000
4,MODIS12004,265,Engelmann spruce,2,916,0.218341,146.119877,56.000000
5,MODIS12004,201,Douglas-fir,120,916,13.100437,90.011329,48.064186
6,MODIS12004,185,Pinyon / juniper woodland,8,916,0.873362,53.870844,21.130435
7,MODIS12004,261,White fir,7,916,0.764192,127.596281,44.835010
8,MODIS12004,182,Rocky Mountain juniper,1,916,0.109170,29.388272,19.714286
9,MODIS12004,971,Deciduous oak woodland,22,916,2.401747,60.104668,34.155633


In [150]:
# Save to file.
out_fp = os.path.join(projdir,'data/tabular/mod/AFD/combined-afd_aspen-fires_TreeMap.csv')
treemap_stats_df.to_csv(out_fp)
print(f"Saved to: {out_fp}")

Saved to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/tabular/mod/AFD/combined-afd_aspen-fires_TreeMap.csv


In [151]:
gc.collect()

31563